In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline
import nltk

In [2]:
# Download required NLTK data
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nikhi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nikhi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nikhi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\nikhi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [3]:
def load_and_preprocess_data():
    """Load and preprocess the Twitter dataset"""
    # column_names = ['target', 'ids', 'date', 'flag', 'user', 'text']
    data = pd.read_csv("IMDB_Dataset.csv", encoding='ISO-8859-1')
    # data['target'] = data['category'].replace(-1, 2)
    return data

In [4]:
#sample data 10,000 rows randomly
data = load_and_preprocess_data()
# data = data.sample(n=30000, random_state=42)

data.shape


(50000, 2)

In [5]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


 """Main function to run the improved sentiment analysis"""

In [6]:
data['target'] = data['sentiment'].replace('positive', 1).replace('negative', 0)

C:\Users\nikhi\AppData\Local\Temp\ipykernel_57012\3214355603.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['target'] = data['sentiment'].replace('positive', 1).replace('negative', 0)


In [7]:
data.head()

,review,sentiment,target
0,One of the other reviewers has mentioned that ...,positive,1
1,A wonderful little production. <br /><br />The...,positive,1
2,I thought this was a wonderful way to spend ti...,positive,1
3,Basically there's a family where a little boy ...,negative,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,1


In [13]:
data.shape

(162980, 3)

In [8]:
data = data.dropna()

In [9]:
# counting the num of missing values in the dataset
data.isnull().sum()

review       0
sentiment    0
target       0
dtype: int64

In [10]:
# checking the distribution of target col

data['target'].value_counts()

target
1    25000
0    25000
Name: count, dtype: int64

In [11]:
def advanced_text_preprocessing(content):

    # Initialize lemmatizer

    lemmatizer = WordNetLemmatizer()
    # Convert to lowercase and remove special characters
    lemmatized_content = re.sub(r'http\S+|www\S+|https\S+', '', content, flags=re.MULTILINE)  # Remove URLs
    lemmatized_content = re.sub(r'@\w+|\#', '', lemmatized_content)  # Remove mentions and hashtag symbols
    lemmatized_content = re.sub(r'[^a-zA-Z\s]', '', lemmatized_content)  # Remove numbers and punctuation
    lemmatized_content = lemmatized_content.lower()

    # Tokenization
    words = word_tokenize(lemmatized_content)
    
    # Remove stopwords and lemmatize
    stop_words = set(stopwords.words('english'))
    # Add custom stopwords that might not be useful for sentiment
    custom_stopwords = {'rt', 'via'}
    stop_words.update(custom_stopwords)
    processed_words = [
        lemmatizer.lemmatize(word) for word in words if word not in stop_words and len(word) > 2
    ]
    lemmatized_content = ' '.join(processed_words)
    return lemmatized_content

In [12]:


data['review'][:3]

0    One of the other reviewers has mentioned that ...
1    A wonderful little production. <br /><br />The...
2    I thought this was a wonderful way to spend ti...
Name: review, dtype: object

In [13]:
 # Apply advanced text preprocessing
    
print("Applying text preprocessing...")
    
data['lemmatized_content'] = data['review'].apply(advanced_text_preprocessing)

Applying text preprocessing...


In [14]:
data['lemmatized_content'][:3]

0    one reviewer mentioned watching episode youll ...
1    wonderful little production filming technique ...
2    thought wonderful way spend time hot summer we...
Name: lemmatized_content, dtype: object

In [15]:
def create_advanced_tfidf():
    """Create an advanced TF-IDF vectorizer"""
    return TfidfVectorizer(
        min_df=5,  # Minimum document frequency
        max_df=0.8,  # Maximum document frequency
        ngram_range=(1, 2),  # Include both unigrams and bigrams
        sublinear_tf=True,  # Apply sublinear scaling to term frequencies
        strip_accents='unicode',
        token_pattern=r'\b\w+\b'  # Only match word characters
    )

In [16]:
def train_model(X_train, y_train, X_test, y_test):
    """Train and evaluate multiple models using GridSearchCV"""
    
    # Create pipelines for different models
    lr_pipeline = Pipeline([
        ('tfidf', create_advanced_tfidf()),
        ('classifier', LogisticRegression())
    ])
    
    rf_pipeline = Pipeline([
        ('tfidf', create_advanced_tfidf()),
        ('classifier', RandomForestClassifier())
    ])
    
    # Define parameter grids for GridSearchCV
    lr_param_grid = {
        'classifier__C': [0.1, 1.0, 10.0],
        'classifier__class_weight': ['balanced', None],
        'classifier__max_iter': [1000]
    }
    
    rf_param_grid = {
        'classifier__n_estimators': [100, 200],
        'classifier__max_depth': [10, 20, None],
        'classifier__class_weight': ['balanced', 'balanced_subsample']
    }
    
    # Perform GridSearchCV for both models
    lr_grid = GridSearchCV(lr_pipeline, lr_param_grid, cv=5, n_jobs=-1, verbose=1)
    rf_grid = GridSearchCV(rf_pipeline, rf_param_grid, cv=5, n_jobs=-1, verbose=1)
    
    # Train both models
    print("Training Logistic Regression...")
    lr_grid.fit(X_train, y_train)
    print("Training Random Forest...")
    rf_grid.fit(X_train, y_train)
    
    # Get best models
    best_lr = lr_grid.best_estimator_
    best_rf = rf_grid.best_estimator_
    
    # Evaluate models
    models = {
        'Logistic Regression': best_lr,
        'Random Forest': best_rf
    }
    
    results = {}
    for name, model in models.items():
        train_pred = model.predict(X_train)
        test_pred = model.predict(X_test)
        
        results[name] = {
            'train_accuracy': accuracy_score(y_train, train_pred),
            'test_accuracy': accuracy_score(y_test, test_pred),
            'classification_report': classification_report(y_test, test_pred)
        }
    
    return results, best_lr if lr_grid.best_score_ > rf_grid.best_score_ else best_rf

# Split data

In [17]:
data.head()

,review,sentiment,target,lemmatized_content
0,One of the other reviewers has mentioned that ...,positive,1,one reviewer mentioned watching episode youll ...
1,A wonderful little production. <br /><br />The...,positive,1,wonderful little production filming technique ...
2,I thought this was a wonderful way to spend ti...,positive,1,thought wonderful way spend time hot summer we...
3,Basically there's a family where a little boy ...,negative,0,basically there family little boy jake think t...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,1,petter matteis love time money visually stunni...


In [18]:
data['lemmatized_content'].values
y = data['target'].values

In [19]:
# Split data
X = data['lemmatized_content'].values
y = data['target'].values
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [20]:
    
# Train and evaluate models
print("Training models...")
results, best_model = train_model(X_train, y_train, X_test, y_test)
    
# Print results
for model_name, metrics in results.items():
    print(f"\n{model_name} Results:")
    print(f"Training Accuracy: {metrics['train_accuracy']:.4f}")
    print(f"Test Accuracy: {metrics['test_accuracy']:.4f}")
    print("\nClassification Report:")
    print(metrics['classification_report'])    
    print("\n")

Training models...
Training Logistic Regression...
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Training Random Forest...
Fitting 5 folds for each of 12 candidates, totalling 60 fits

Logistic Regression Results:
Training Accuracy: 0.9923
Test Accuracy: 0.9079

Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.90      0.91      5000
           1       0.90      0.91      0.91      5000

    accuracy                           0.91     10000
   macro avg       0.91      0.91      0.91     10000
weighted avg       0.91      0.91      0.91     10000




Random Forest Results:
Training Accuracy: 1.0000
Test Accuracy: 0.8728

Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.87      0.87      5000
           1       0.87      0.88      0.87      5000

    accuracy                           0.87     10000
   macro avg       0.87      0.87      0.87     10000
wei

In [58]:
import pickle

In [59]:
filename = 'twitter_model_lemit.pkl'
pickle.dump(best_model, open(filename, 'wb'))

In [61]:
# loading the saved model

loaded_model = pickle.load(open('twitter_model_lemit.pkl', 'rb'))

In [63]:
X_new = X_test[5]


In [65]:
print(y_test[5])


0


In [69]:
prediction = (loaded_model.predict([X_new]))

print(prediction)

if (prediction[0]==0):
    print('Negative tweet')
else:
    print('Positive tweet')



[0]
Negative tweet


# Sentiment Analysis Model Performance Report

## Model Performance Overview

| Metric | Logistic Regression | Random Forest |
|--------|---------------------|---------------|
| **Overall Accuracy** | 90.79% | 87.28% |
| **Training Accuracy** | 99.23% | 100.00% |
| **Precision (Negative)** | 0.91 | 0.88 |
| **Precision (Positive)** | 0.90 | 0.87 |
| **Recall (Negative)** | 0.90 | 0.87 |
| **Recall (Positive)** | 0.91 | 0.88 |
| **F1-Score (Negative)** | 0.91 | 0.87 |
| **F1-Score (Positive)** | 0.91 | 0.87 |

## Key Findings and Technical Specifications

### Data Preprocessing
- Dataset: IMDB Movie Reviews
- Preprocessing Techniques:
  - URL and special character removal
  - Text lemmatization
  - Stopwords elimination
  - Custom text normalization

### Model Configuration
- Vectorization: TF-IDF with advanced configurations
  - Unigrams and bigrams
  - Minimum document frequency: 5
  - Maximum document frequency: 0.8
- Cross-validation: 5-fold stratified

### Model Performance Insights
- Logistic Regression slightly outperforms Random Forest
- Both models show robust performance with >87% accuracy
- Minimal variance between training and test accuracies indicates good generalization

## Challenges and Observations
- Potential overfitting in Random Forest (100% training accuracy)
- Balanced performance across positive and negative classes
- Effective preprocessing mitigates noise in textual data

## Recommendations
1. Consider ensemble methods to potentially improve accuracy
2. Experiment with deeper text preprocessing techniques
3. Evaluate model on out-of-domain datasets for true generalizability

## Conclusion
The Logistic Regression model provides the most balanced performance for sentiment analysis, with a test accuracy of 90.79% and consistent precision and recall across sentiment classes.